# TODO

----------------> here
1. run italy
2. run processing and images
3. move images to other folder
4. dont clear df
5. run other countries
6. run processing and images
7. move images to other folder

# Parameters

In [74]:
# starting url
param_url = "https://franklin.library.upenn.edu/catalog/FRANKLIN_9940439813503681"

# output image folder, excel file
param_output_folder = "images/" # please create this folder before running this program
param_output_excel_file = "result.csv"

# fields
param_fields = ['source', 'filename', 'artist', 'title', 'date', 'full-date', 'period', 'geography', 'country','url']
param_fields.append('image_url')

# miscellaneous
param_source = "artstor"

# periods
param_periods = []
param_periods.append({ "start": 1400,  "end": 1495, "period": "1400-1495" })

# image size
LARGE = 'LARGE'
MEDIUM = 'MEDIUM'
param_image_size = LARGE

# countries
param_areas = []
param_areas.append({ "country": "Italy",          "geography": "Italy"  })
# param_areas.append({ "country": "Austria",        "geography": "Europe" })
# param_areas.append({ "country": "Belgium",        "geography": "Europe" })
# param_areas.append({ "country": "Czech Republic", "geography": "Europe" })
# param_areas.append({ "country": "France",         "geography": "Europe" })
# param_areas.append({ "country": "Germany",        "geography": "Europe" })
# param_areas.append({ "country": "Netherlands",    "geography": "Europe" })
# param_areas.append({ "country": "Switzerland",    "geography": "Europe" })
# param_areas.append({ "country": "United Kingdom", "geography": "Europe" })
# param_areas.append({ "country": "Spain",          "geography": "Europe" })
# param_areas.append({ "country": "Portugal",       "geography": "Europe" })
# param_areas.append({ "country": "Greece",         "geography": "Europe" })

DOWNLOAD_DIRECTORY = "/Users/Kevin/Downloads"

In [75]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import re
import wget
import os
import xlrd
import hashlib

# Start program

In [76]:
# Start selenium
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get(param_url)

In [77]:
# # Visit artstor
# artstor_url = "https://library-artstor-org.proxy.library.upenn.edu/#/search/*;artclassification_str=Paintings"
# driver.get(artstor_url)
# time.sleep(3)

In [78]:
# # check username and password is added to environmental variables
# username = os.environ.get('CANVAS_USERNAME')
# password = os.environ.get('CANVAS_PASSWORD')
# if username is None:
#     raise ValueError("Username is empty. Please add \"CANVAS_USERNAME\" to your environmental variables")
    
# if password is None:
#     raise ValueError("Password is empty. Please add \"CANVAS_PASSWORD\" to your environmental variables")  

# # log into penn
# search_box = driver.find_element_by_name('login')
# search_box.send_keys(username)
# search_box = driver.find_element_by_name('password')
# search_box.send_keys(password)
# search_box.submit()
# time.sleep(10)

In [79]:
# check username and password is added to environmental variables

artstor_email = os.environ.get('ARTSTOR_EMAIL')
artstor_password = os.environ.get('ARTSTOR_PASSWORD')

if artstor_email is None:
    raise ValueError("Username is empty. Please add \"ARTSTOR_EMAIL\" to your environmental variables")
    
if artstor_password is None:
    raise ValueError("Password is empty. Please add \"ARTSTOR_PASSWORD\" to your environmental variables")  

driver.get("https://library.artstor.org/#/login")

time.sleep(1)
driver.find_element_by_id("inputLoginEmail").send_keys(artstor_email)
driver.find_element_by_id("inputLoginPassword").send_keys(artstor_password)

driver.find_element_by_id("btnLoginRemote").click()
time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="inputLoginEmail"]"}
  (Session info: chrome=75.0.3770.142)


In [ ]:
# Visit artstor again
driver.get("https://library.artstor.org/#/search/*;artclassification_str=Paintings")
time.sleep(3)

# click 72 per page 

In [ ]:
# get df
df = pd.DataFrame(columns=param_fields)
df

In [ ]:
# parse param periods
divided_periods = []
for param_period in param_periods:
    start_date = param_period['start']
    end_date = param_period['end']
    cur_start_date = start_date
    while cur_start_date <= end_date:
        cur_end_date = min(cur_start_date + 9, end_date)
        divided_periods.append({ "start": cur_start_date, "end": cur_end_date, "period": param_period['period'] })
        cur_start_date = cur_end_date + 1
divided_periods

# Functions

### Function: get excel file

In [ ]:
# create result excel file if it does not exist
def get_excel_file(output_excel_file, fields):
    if os.path.exists(output_excel_file): 
        return pd.read_csv(output_excel_file, index_col=0)
    else:
        return pd.DataFrame(columns=fields)

In [ ]:
def get_existing_file_count(output_folder):
    files = os.listdir(output_folder)
    return len(files)

### Function: process one country

In [ ]:
param_fields = ['source', 'filename', 'artist', 'title', 'date', 'full-date', 'period', 'geography', 'country']
param_fields.append('image_url')
param_output_folder = "images/" # please create this folder before running this program
param_output_excel_file = "result.csv"
param_source = "artstor"
param_period="1496-1520"
param_geography="Italy"
param_country="Italy"


def process_one_country(driver, df, source, fields, period, geography, country):
    while True:
        cards = driver.find_elements_by_class_name("card--asset")
        for card in cards:
            i = len(df)
            df.loc[i,'source'] = source
            try:
                df.loc[i,'artist'] = card.find_element_by_class_name("asset-agent").text
            except:
                pass

            try:
                df.loc[i,'full-date'] = card.find_element_by_class_name("asset-date").text
            except:
                pass

            try:
                df.loc[i,'image_url'] = card.find_element_by_tag_name("img").get_attribute("src")
            except:
                pass

            try:
                df.loc[i,'title'] = card.find_element_by_class_name("asset-title").text
            except:
                pass
            
            try:
                df.loc[i,'url'] = card.get_attribute("href")
            except:
                pass

            df.loc[i,'period'] = period
            df.loc[i,'geography'] = geography
            df.loc[i,'country'] = country

        # click next button
        try:
            driver.find_element_by_xpath("//button[@title='Next']").click()
        except Exception as e: 
            break
        time.sleep(3)
    return df

### Function: parse 

In [ ]:
NO_DATE = 0
FAILURE = -1
QUESTIONABLE = -2
NO_RESULT = -3
questionables = []
legals = ['Not dated','Not Dated','not dated','n. d.','n.d.','o.J.','early work']
illegals = ['B.C','BC']
def process_special_dates(full_date):
    for illegal in illegals:
        if illegal in full_date:
            return FAILURE
        
    for legal in legals:
        if legal in full_date:
            return NO_DATE
        
    for q in questionables:
        if q in full_date:
            return QUESTIONABLE
        
    if 'Fifteenth century' in full_date or 'fifteenth century' in full_date:
        return 1500
    return NO_RESULT
    
def process_parsing(df):    
    # drop nans
    df = delete_na(df,['full-date']);
    
    # parse date from full-date
    for index, row in df.iterrows():
        # pattern: no patterns
        res = process_special_dates(row['full-date'])
        if res != NO_RESULT:
            row['date'] = res
            continue
            
        # pattern: XXXX
        arr = re.findall('\d{4}', str(row['full-date']))
        if len(arr) > 1 and 'restored' not in row['full-date']:
            row['date'] = (int(arr[0]) + int(arr[1])) / 2    
            continue
        if len(arr) == 1:
            row['date'] = int(arr[0])
            continue
        
        # pattern: XXth
        arr = re.findall('\d{2}', str(row['full-date']))
        if len(arr) > 1 and 'restored' not in row['full-date']:
            d1 = int(arr[0]) * 100 - 50
            d2 = int(arr[1]) * 100 - 50
            row['date'] = (d1 + d2) / 2
            continue
        if len(arr) > 0:
            row['date'] = int(arr[0]) * 100 - 50
            continue
            
        print("can't parse",row['full-date'])
                
    return df


# For testing
# df2 = df.copy()
# df2 = process_parsing(df2)
# df2

### Function: download images

In [ ]:
def process_small_images(df, output_folder, seconds):
    # download images
    for index, row in df.iterrows():
        filename = hashlib.md5(row['image_url'].encode()).hexdigest() + '.jpg'
        df.at[index,'filename'] = filename
        filename = output_folder + filename
        url = row['image_url']
        if not os.path.exists(filename):
            try:
                wget.download(url, filename)
                time.sleep(seconds) # this is for being a polite visitor
            except:
                print("Can't download", filename, "using:", url)
        
    return df.reset_index(drop=True)

def delete_na(df, cols):
    df = df.dropna(subset=cols)
    df = df.reset_index(drop=True)
    return df


### Clicks

In [14]:
def click_to_72_per_page(driver):
    driver.find_element_by_class_name("icon-large-view").click()

def clear_country(driver):
    try:
        xpath_rule = "//a[@aria-label='Clear, click to clear all selected Geography filters']"
        driver.find_element_by_xpath(xpath_rule).click()
    except:
        pass

def click_expand_europe(driver):
    xpath_rule = "//i[contains(@aria-label,'Expand Europe') and contains(@class,'icon-direction')]"
    driver.find_element_by_xpath(xpath_rule).click()
    
def click_country(driver, country):
    click_expand_europe(driver)
    time.sleep(1)
    xpath_rule = "//button[contains(@class,'filter') and contains(text(),'{}')]".format(country)
    driver.find_element_by_xpath(xpath_rule).click()

def click_sort_dropdown(driver):
    driver.find_element_by_id('sortdropdownMenu').click()
    
def click_sort_by_date(driver):
    try:
        click_sort_dropdown(driver)
        time.sleep(0.5)
        xpath_rule = "//button[contains(@class,'dropdown-item') and contains(text(),'Date')]"
        driver.find_element_by_xpath(xpath_rule).click()
    except:
        click_sort_dropdown(driver)

def click_ce(driver):
    driver.find_element_by_id('buttonDateFilterStartCE').click()

def click_input_apply_button(driver):
    driver.find_element_by_id('buttonDateFilterSubmit').click()

def clear_date(driver):
    try:
        driver.find_element_by_id('buttonDateFilterClear').click()
    except:
        pass
    
def input_dates(driver,start_date,end_date):
    click_ce(driver)
    time.sleep(0.5)
    xpath_rule_start = "//input[contains(@aria-label,'Start date')]"
    xpath_rule_end   = "//input[contains(@aria-label,'End date')]"
    start_input = driver.find_element_by_xpath(xpath_rule_start)
    end_input   = driver.find_element_by_xpath(xpath_rule_end)
    start_input.clear()
    time.sleep(0.2)
    start_input.send_keys(start_date)
    time.sleep(0.2)
    end_input.clear()
    time.sleep(0.2)
    end_input.send_keys(end_date)
    time.sleep(0.2)
    click_input_apply_button(driver)


# Main program

In [26]:
# run through all country, all periods
for area in param_areas:
    print(area)
    for d_period in divided_periods:
        click_to_72_per_page(driver) # buggy
        time.sleep(1)
        clear_country(driver)
        time.sleep(5)
        try:
            click_country(driver,area['country'])
            time.sleep(5)
        except:
            continue
            
        click_sort_by_date(driver)
        time.sleep(5)
        input_dates(driver, d_period['start'], d_period['end'])
        df = process_one_country(driver, df, param_source, param_fields, d_period['period'], area['geography'], area['country'])
        
        clear_date(driver)
        time.sleep(5)
        print(d_period['end'], "({})".format(len(df)), end=", ")
        print()


{'country': 'Italy', 'geography': 'Italy'}
1409 (0), 
1419 (212), 
1429 (705), 
1439 (1311), 
1449 (2232), 
1459 (2930), 
1469 (3675), 
1479 (4612), 
1489 (5881), 
1495 (6840), 


In [62]:
for index, row in df.iterrows():
    print(row['url'])


In [28]:
df2 = df.copy()
df2 = df.astype({"filename": object})

# parse and download images
df2 = process_parsing(df2)

# drop illegal dates
print(len(df2))
df2 = df2[df2['date'] >= 0] 

# drop out of range
print(len(df2))
df2['date'] = df2['date'].astype('int64')
r_start = param_periods[0]['start']
r_end = param_periods[0]['end']
legal_index = df2['date'] != 0 & ((df2['date'] < r_start) | (df2['date'] > r_end))
df2 = df2[legal_index]

# drop null artists or image_urls
print(len(df2))
df2 = delete_na(df2,['artist','image_url'])

# drop image_url duplicates
print(len(df2))
df2 = df2.drop_duplicates(subset=['image_url'])
print(len(df2))

6607
5959
5894
5850
5776


In [37]:
df2 = df2.astype({'filename': object})
df2 = df2.fillna('')

,source,filename,artist,title,date,full-date,period,geography,country,image_url,url
0,artstor,,"Salimbeni, Lorenzo, 1374-ca.1420","Urbino, Oraorio di S. Giovanni Madonna in Para...",1416,1416,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/ARTSTOR_10...
1,artstor,,"Salimbeni, Lorenzo, 1374-ca.1420","Urbino, Oraorio di S. Giovanni Scenes from Lif...",1416,1416,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/ARTSTOR_10...
2,artstor,,"Lorenzo, Monaco, 1370 or 71-1425",Agony in the Garden: det.,1405,c.1405-6,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/ARTSTOR_10...
3,artstor,,Lorenzo Salimbeni and Jacopo Salimbeni,Scenes from the Life of Saint John the Baptist...,1416,c. 1416,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/SCALA_ARCH...
4,artstor,,Lorenzo Salimbeni and Jacopo Salimbeni,Scenes from the Life of Saint John the Baptist...,1416,c. 1416,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/SCALA_ARCH...
5,artstor,,"Lorenzo, Monaco, 1370 or 71-1425",Agony in the Garden: det.,1405,c.1405-6,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/ARTSTOR_10...
6,artstor,,"Lorenzo, Monaco, 1370 or 71-1425",Agony in the Garden,1405,c.1405-6,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/ARTSTOR_10...
7,artstor,,"Lorenzo, Monaco, 1370 or 71-1425",Agony in the Garden: det.,1405,c.1405-6,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/ARTSTOR_10...
8,artstor,,Lorenzo Salimbeni and Jacopo Salimbeni,Crucifixion; detail of Mourners Under the Cross,1416,c. 1416,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/SCALA_ARCH...
9,artstor,,Lorenzo Salimbeni and Jacopo Salimbeni,Scenes from the Life of Saint John the Baptist...,1416,c. 1416,1400-1495,Italy,Italy,https://mdxdv.artstor.org/thumb/imgstor/size2/...,https://library.artstor.org/#/asset/SCALA_ARCH...


In [110]:
if param_image_size == 'MEDIUM':
    df3 = process_medium_images(df2, param_output_folder, 0.5)
    df3 = process_medium_images(df2, param_output_folder, 0.5)
    df3 = process_medium_images(df2, param_output_folder, 0.5)


if param_image_size == 'LARGE':
    for k in range(0, 10):
        for index, row in df2.iterrows():
            if index % 500 == 0:
                print("processing", index)

            # skip already downloaded
            filename = hashlib.md5(row['image_url'].encode()).hexdigest() + '.jpg'
            if os.path.isfile("images/" + filename):
                continue

            driver.get(row['url'])
            time.sleep(1)
            driver.find_element_by_id("downloadAssetDropdown").click()
            driver.find_element_by_id("downloadAssetLink").click()
            try:
                time.sleep(1)
                driver.find_element_by_xpath("//a[contains(text(),'accept')]").click()
                time.sleep(5)
            except:
                pass

            for i in range(0,3):
                time.sleep(1)
                downloaded_files = os.listdir(DOWNLOAD_DIRECTORY)
                jpg_files = list(filter(lambda x: x.endswith(".jpg") , downloaded_files))
                if len(jpg_files) == 1:
                    os.rename(DOWNLOAD_DIRECTORY + "/" + jpg_files[0], "images/" + filename)
                    df2.at[index,'filename'] = filename
                    break
                elif len(jpg_files) > 1:
                    print("too many files", index)
                    input("Press Enter to continue...")
                time.sleep(1)



processing 0
processing 500
processing 1000
processing 1500
processing 2000
processing 2500
processing 3000
processing 3500
processing 4000
processing 4500
processing 5000
processing 5500
processing 0
processing 500
processing 1000
processing 1500
processing 2000
processing 2500
processing 3000
processing 3500
processing 4000
processing 4500
processing 5000
processing 5500
processing 0
processing 500
processing 1000
processing 1500
processing 2000
processing 2500
processing 3000
processing 3500
processing 4000
processing 4500
processing 5000
processing 5500
processing 0
processing 500
processing 1000
processing 1500
processing 2000
processing 2500
processing 3000
processing 3500
processing 4000
processing 4500
processing 5000
processing 5500
processing 0
processing 500
processing 1000
processing 1500
processing 2000
processing 2500
processing 3000
processing 3500
processing 4000
processing 4500
processing 5000
processing 5500
processing 0
processing 500
processing 1000
processing 1500


In [ ]:
# move image location
# italy_df = df3[df3['geography'] == 'Italy']
# other_df = df3[df3['geography'] != 'Italy']

# for index, row in italy_df.iterrows():
#     os.rename(param_output_folder + row['filename'], "Italy/" + row['filename'])

# for index, row in other_df.iterrows():
#     os.rename(param_output_folder + row['filename'], "Western_Europe/" + row['filename'])

In [123]:
# save to excel
# df3.to_csv(param_output_excel_file)
df3.to_csv(param_output_excel_file)

In [112]:
count = 0
for index, row in df2.iterrows():
    filename = hashlib.md5(row['image_url'].encode()).hexdigest() + '.jpg'
    if os.path.isfile("images/" + filename):
        continue
    else:
        print(index+1, row['url'])
        count = count + 1
print(count)

587 https://library.artstor.org/#/asset/FRICKIG_10310492385;prevRouteTS=1564418371945
840 https://library.artstor.org/#/asset/FRICKIG_10310849006;prevRouteTS=1564418433189
1597 https://library.artstor.org/#/asset/NGA_REALI_1039655483;prevRouteTS=1564418698936
1641 https://library.artstor.org/#/asset/NGA_REALI_1039655581;prevRouteTS=1564418714499
1745 https://library.artstor.org/#/asset/NGA_REALI_1039655274;prevRouteTS=1564418730015
1746 https://library.artstor.org/#/asset/NGA_REALI_1039654910;prevRouteTS=1564418730015
2049 https://library.artstor.org/#/asset/FRICKIG_1039776287;prevRouteTS=1564418891536
2624 https://library.artstor.org/#/asset/FRICKIG_10310500865;prevRouteTS=1564419101676
2887 https://library.artstor.org/#/asset/NGA_REALI_1039655613;prevRouteTS=1564419165173
3116 https://library.artstor.org/#/asset/FRICKIG_1039775392;prevRouteTS=1564419308236
3398 https://library.artstor.org/#/asset/FRICKIG_10310492749;prevRouteTS=1564419372865
3432 https://library.artstor.org/#/asset/F